In [ ]:
install.packages("igraph")
library('igraph')

In [ ]:
if (!require("clevr")) install.packages("clevr")
library ("clevr")

**Import the finance data**

In [ ]:
path = "finance_data/data/"
out.file<-""
file.names <- dir(path, pattern =".csv")
m <- matrix(, nrow = 0, ncol = 765)
sectors.table <- read.table("finance_data/Name_sector.csv",header=TRUE, sep=",", stringsAsFactors=TRUE)
sectors = c()
sectors.names <- c()
for(i in 1:length(file.names)){
    file <- read.table(paste("finance_data/data/",file.names[i],sep = ""),header=TRUE, sep=",", stringsAsFactors=FALSE)
    if(length(file$Close)==765){
        m <- rbind(m, matrix(file$Close, nrow=1, ncol=765))
        mystr <- substr(file.names[i], 1, nchar(file.names[i])-4)
        sector <- sectors.table$Sector[which(sectors.table$Symbol == mystr)]
        sectors.names = c(sectors.names, toString(sector))
    }
}
com_num = length(sectors.names)
data_num = ncol(m)
sector.set<-as.factor(sectors.names)
sectors.index<-as.numeric(sector.set)
num_sector <- length(unique(sectors.index))
table(sector.set)

In [ ]:
ticker_sector = read.csv("finance_data/Name_sector.csv", header=TRUE, stringsAsFactors=FALSE)
unique_sector = unique(ticker_sector[,2])
print(sprintf("The Number of Companies in The Ticker List is: %s", nrow(ticker_sector)))
print(sprintf("The Number of Unique Sectors: %s", length(unique_sector)))

In [ ]:
# the function for calculate the correlation
calculate_correlation <- function(edge_file, log_return, ticker_sector)
{
  cat("start", "\t", "end", "\t", "weight", file=edge_file)
  number_of_comp = dim(ticker_sector)[1]
  for(tempI in c(1:(number_of_comp-1))){
      for(tempJ in c((tempI+1):number_of_comp)){
          r_i = mean(log_return[tempI,])
          r_j = mean(log_return[tempJ,])
          r_ij = mean(log_return[tempI,] * log_return[tempJ,])
          r_i_squared = log_return[tempI,]^2
          r_j_squared = log_return[tempJ,]^2
          p_ij = (r_ij - (r_i * r_j)) / (sqrt((mean(r_i_squared)-(r_i^2)) * (mean(r_j_squared)-(r_j^2))))
          w_ij = sqrt(2 * (1-p_ij))
          cat('\n', ticker_sector[tempI, 1], '\t', ticker_sector[tempJ, 1], '\t', w_ij, file=edge_file)
      }
   }
}

**Question 2 - 5 for DAILY data**


QUESTION 2: Plot a histogram showing the un-normalized distribution of edge weights.

In [ ]:
# obtain the file names from the finance data
filenames <- paste("finance_data/data", list.files("finance_data/data", pattern="*.csv"), sep="/")
# generate a empty array to store the companies with missing data
miss_data_comp <- c()
# generate a empty array to stor the log return
log_return <- matrix(0, length(filenames), 764)

for (tempIndex in c(1:length(filenames)))
{
  # store the current company data
  temp_comp_data <- read.csv(filenames[tempIndex], header=TRUE, stringsAsFactors=FALSE)
  # judge whether the company has some missing data through checking the dimension
  if(dim(temp_comp_data)[1] != 765)
  {
    miss_data_comp <- append(miss_data_comp, tempIndex)
  }
  else
  {
    p_i <- temp_comp_data[,5]
    q_i <- c()
    r_i <- c()
    for (tempT in c(2:length(p_i)))
    {
      q_i[tempT-1] <- (p_i[tempT] - p_i[tempT-1])/p_i[tempT - 1]
    }
    r_i = log(q_i + 1)
    log_return[tempIndex,] = r_i
  }
}
ticker_sector <- ticker_sector[-miss_data_comp,]
log_return <- log_return[-miss_data_comp,]

In [ ]:
edge_file <- file("finance_data/edges_date_daily.txt", "w")
calculate_correlation(edge_file, log_return, ticker_sector)
close(edge_file)

In [ ]:
edge_list = read.delim("finance_data/edges_date_daily.txt", header=TRUE)
graph_frame = graph.data.frame(edge_list, directed = FALSE)
E(graph_frame)$weight = edge_list[,"weight"]
hist(E(graph_frame)$weight, col="orange", main="The  Distribution of Un-Normalized Edge Weights for Daily",xlab="The Edge Weights", ylab="The Frequency of Edge Weights")

QUESTION 3: Extract the MST of the correlation graph. Each stock can be categorized into a sector, which can be found in Name sector.csv file. Plot the MST and color-code the nodes
based on sectors. Do you see any pattern in the MST? The structures that you find in MST are called Vine clusters. Provide a detailed explanation about the pattern you observe

In [ ]:
# generate a empty array to stor the index of each sector
sector_index <- c()
colorPalette <- rainbow(length(unique_sector), s = 1, v = 1)
for(tempIndex in c(1:vcount(graph_frame))) 
{
    sector_index = append(sector_index, which(unique_sector == ticker_sector[tempIndex, 2]))
}
MST <- mst(graph_frame, algorithm="prim")

# Set a larger plot size
options(repr.plot.width = 8, repr.plot.height = 12)

plot(MST, vertex.size=3, vertex.label=NA, vertex.color=colorPalette[sector_index])

# Custom legend with color squares and names
legend_colors <- colorPalette
legend_names <- unique_sector
legend(x=-1,y = -1, legend = legend_names, fill = legend_colors, bty = "n", ncol=2, cex = 1)

par(xpd=TRUE)
title("The MST of The Correlation Graph")



QUESTION 4: Run a community detection algorithm (for example walktrap) on the MST obtained above. Plot the communities formed. Compute the homogeneity and completeness of the clustering. (you can use the ’clevr’ library in r to compute homogeneity and completeness).

In [ ]:
graph_community = cluster_walktrap(MST)
# Set a larger plot size
options(repr.plot.width = 8, repr.plot.height = 8)
plot(MST, mark.groups=graph_community, edge.width=0.5,
     vertex.size=3, vertex.color=graph_community$membership, vertex.label="",
     main="The Community Structure for The MST")

In [ ]:
# homogeneity and completeness
# figure out true and pred
true <- c(sector_index)
pred <- c(graph_community$membership)
print(sprintf("The homogenety for Daily data: %s",homogeneity(true, pred)))
print(sprintf("The completness for Daily data: %s",completeness(true,pred)))

QUESTION 5: Report the value of α for the above two cases and provide an interpretation for the difference.

In [ ]:
# generate the function for calculating the alpha
calculate_alpha <- function(graph, log_return, ticker_sector, unique_sector)
{
  # generate the sector
  Sector = c()
  for (tempIndex in c(1:length(unique_sector)))
  {
    Sector[tempIndex] = length(which(unique_sector[tempIndex] == ticker_sector[,2]))
  }
  # generate the empty array for case one p
  Pcase_1 <- c()
  # generate the empty array for case two p
  Pcase_2 <- c()
  for(tempV in c(1:vcount(graph)))
  {
    neighbors_tempV = neighbors(graph, tempV)
    N_i <- length(neighbors_tempV) 
    Q_i <- 0
    for(tempIndex in neighbors_tempV)
    {
      if(ticker_sector[tempIndex,2]==ticker_sector[tempV,2])
      {
        Q_i = Q_i + 1
      }
    }
    Pcase_1[tempV] = Q_i/N_i
    Pcase_2[tempV] = Sector[which(unique_sector==ticker_sector[tempV,2])]/vcount(graph)
  } 
  alpha_case_1 = sum(Pcase_1)/vcount(graph)
  alpha_case_2 = sum(Pcase_2)/vcount(graph)
  print(sprintf("The Alpga Value for The First Case: %s", alpha_case_1))
  print(sprintf("The Alpga Value for The Second Case: %s", alpha_case_2))
}

In [ ]:
calculate_alpha(MST,log_return,ticker_sector,unique_sector)

**Question 2 - 5 for WEEKLY data**

In [ ]:
ticker_sector = read.csv("finance_data/Name_sector.csv", header=TRUE, stringsAsFactors=FALSE)
unique_sector = unique(ticker_sector[,2])
print(sprintf("The Number of Companies in The Ticker List is: %s", nrow(ticker_sector)))
print(sprintf("The Number of Unique Sectors: %s", length(unique_sector)))

QUESTION 6: Repeat questions 2,3,4,5 on the WEEKLY data.

QUESTION 2: Plot a histogram showing the un-normalized distribution of edge weights.

In [ ]:
# obtain the file names from the finance data
filenames <- paste("finance_data/data", list.files("finance_data/data", pattern="*.csv"), sep="/")
# generate a empty array to store the companies with missing data
miss_data_comp <- c()
# generate a empty array to stor the log return
log_return <- matrix(0, length(filenames), 142)

for (tempIndex in c(1:length(filenames)))
{
  # store the current company data
  temp_comp_data <- read.csv(filenames[tempIndex], header=TRUE, stringsAsFactors=FALSE)
  temp_comp_data["Day"] <- weekdays(as.Date(temp_comp_data[,1]))
  temp_comp_data <- subset(temp_comp_data, Day == 'Monday')
  # judge whether the company has some missing data through checking the dimension
  if(dim(temp_comp_data)[1] != 143)
  {
    miss_data_comp <- append(miss_data_comp, tempIndex)
  }
  else
  {
    p_i <- temp_comp_data[,5]
    q_i <- c()
    r_i <- c()
    for (tempT in c(2:length(p_i)))
    {
      q_i[tempT-1] <- (p_i[tempT] - p_i[tempT-1])/p_i[tempT - 1]
    }
    r_i = log(q_i + 1)
    log_return[tempIndex,] = r_i
  }
}
ticker_sector <- ticker_sector[-miss_data_comp,]
log_return <- log_return[-miss_data_comp,]

In [ ]:
edge_file <- file("finance_data/edges_date_weekly.txt", "w")
calculate_correlation(edge_file, log_return, ticker_sector)
close(edge_file)

In [ ]:
edge_list = read.delim("finance_data/edges_date_weekly.txt", header=TRUE)
graph_frame = graph.data.frame(edge_list, directed = FALSE)
E(graph_frame)$weight = edge_list[,"weight"]
hist(E(graph_frame)$weight, col="orange", main="The  Distribution of Un-Normalized Edge Weights for Weekly",xlab="The Edge Weights", ylab="The Frequency of Edge Weights")

QUESTION 3: Extract the MST of the correlation graph. Each stock can be categorized into a sector, which can be found in Name sector.csv file. Plot the MST and color-code the nodes
based on sectors. Do you see any pattern in the MST? The structures that you find in MST are called Vine clusters. Provide a detailed explanation about the pattern you observe

In [ ]:
# generate a empty array to stor the index of each sector
sector_index <- c()
colorPalette <- rainbow(length(unique_sector), s = 1, v = 1)
for(tempIndex in c(1:vcount(graph_frame))) 
{
    sector_index = append(sector_index, which(unique_sector == ticker_sector[tempIndex, 2]))
}
MST <- mst(graph_frame, algorithm="prim")

# Set a larger plot size
options(repr.plot.width = 8, repr.plot.height = 12)

plot(MST, vertex.size=3, vertex.label=NA, vertex.color=colorPalette[sector_index])

# Custom legend with color squares and names
legend_colors <- colorPalette
legend_names <- unique_sector
legend(x=-1,y = -1, legend = legend_names, fill = legend_colors, bty = "n", ncol=2, cex = 1)

par(xpd=TRUE)
title("The MST of The Correlation Graph")

QUESTION 4: Run a community detection algorithm (for example walktrap) on the MST obtained above. Plot the communities formed. Compute the homogeneity and completeness of the clustering. (you can use the ’clevr’ library in r to compute homogeneity and completeness).

In [ ]:
graph_community = cluster_walktrap(MST)
# Set a larger plot size
options(repr.plot.width = 8, repr.plot.height = 8)
plot(MST, mark.groups=graph_community, edge.width=0.5,
     vertex.size=3, vertex.color=graph_community$membership, vertex.label="",
     main="The Community Structure for The MST")

In [ ]:
# homogeneity and completeness
# figure out true and pred
true <- c(sector_index)
pred <- c(graph_community$membership)
print(sprintf("The homogenety for Daily data: %s",homogeneity(true, pred)))
print(sprintf("The completness for Daily data: %s",completeness(true,pred)))

QUESTION 5: Report the value of α for the above two cases and provide an interpretation for the difference.

In [ ]:
calculate_alpha(MST,log_return,ticker_sector,unique_sector)

**Question 2 - 5 for MONTHLY data**

In [ ]:
ticker_sector = read.csv("finance_data/Name_sector.csv", header=TRUE, stringsAsFactors=FALSE)
unique_sector = unique(ticker_sector[,2])
print(sprintf("The Number of Companies in The Ticker List is: %s", nrow(ticker_sector)))
print(sprintf("The Number of Unique Sectors: %s", length(unique_sector)))

QUESTION 7: Repeat questions 2,3,4,5 on the MONTHLY data.

QUESTION 2: Plot a histogram showing the un-normalized distribution of edge weights.

In [ ]:
# obtain the file names from the finance data
filenames <- paste("finance_data/data", list.files("finance_data/data", pattern="*.csv"), sep="/")
# generate a empty array to store the companies with missing data
miss_data_comp <- c()
# generate a empty array to stor the log return
log_return <- matrix(0, length(filenames), 24)

for (tempIndex in c(1:length(filenames)))
{
  # store the current company data
  temp_comp_data <- read.csv(filenames[tempIndex], header=TRUE, stringsAsFactors=FALSE)
  temp_comp_data["Day"] <- format(as.Date(temp_comp_data[,1]), format = "%d")
  temp_comp_data <- subset(temp_comp_data, Day == '15')
  # judge whether the company has some missing data through checking the dimension
  if(dim(temp_comp_data)[1] != 25)
  {
    miss_data_comp <- append(miss_data_comp, tempIndex)
  }
  else
  {
    p_i <- temp_comp_data[,5]
    q_i <- c()
    r_i <- c()
    for (tempT in c(2:length(p_i)))
    {
      q_i[tempT-1] <- (p_i[tempT] - p_i[tempT-1])/p_i[tempT - 1]
    }
    r_i = log(q_i + 1)
    log_return[tempIndex,] = r_i
  }
}
ticker_sector <- ticker_sector[-miss_data_comp,]
log_return <- log_return[-miss_data_comp,]

In [ ]:
edge_file <- file("finance_data/edges_date_monthly.txt", "w")
calculate_correlation(edge_file, log_return, ticker_sector)
close(edge_file)

In [ ]:
edge_list = read.delim("finance_data/edges_date_monthly.txt", header=TRUE)
graph_frame = graph.data.frame(edge_list, directed = FALSE)
E(graph_frame)$weight = edge_list[,"weight"]
hist(E(graph_frame)$weight, col="orange", main="The  Distribution of Un-Normalized Edge Weights for Weekly",xlab="The Edge Weights", ylab="The Frequency of Edge Weights")

QUESTION 3: Extract the MST of the correlation graph. Each stock can be categorized into a sector, which can be found in Name sector.csv file. Plot the MST and color-code the nodes
based on sectors. Do you see any pattern in the MST? The structures that you find in MST are called Vine clusters. Provide a detailed explanation about the pattern you observe

In [ ]:
# generate a empty array to stor the index of each sector
sector_index <- c()
colorPalette <- rainbow(length(unique_sector), s = 1, v = 1)
for(tempIndex in c(1:vcount(graph_frame))) 
{
    sector_index = append(sector_index, which(unique_sector == ticker_sector[tempIndex, 2]))
}
MST <- mst(graph_frame, algorithm="prim")

# Set a larger plot size
options(repr.plot.width = 8, repr.plot.height = 12)

plot(MST, vertex.size=3, vertex.label=NA, vertex.color=colorPalette[sector_index])

# Custom legend with color squares and names
legend_colors <- colorPalette
legend_names <- unique_sector
legend(x=-1,y = -1, legend = legend_names, fill = legend_colors, bty = "n", ncol=2, cex = 1)

par(xpd=TRUE)
title("The MST of The Correlation Graph")

QUESTION 4: Run a community detection algorithm (for example walktrap) on the MST obtained above. Plot the communities formed. Compute the homogeneity and completeness of the clustering. (you can use the ’clevr’ library in r to compute homogeneity and completeness).

In [ ]:
graph_community = cluster_walktrap(MST)
# Set a larger plot size
options(repr.plot.width = 8, repr.plot.height = 8)
plot(MST, mark.groups=graph_community, edge.width=0.5,
     vertex.size=3, vertex.color=graph_community$membership, vertex.label="",
     main="The Community Structure for The MST")

In [ ]:
# homogeneity and completeness
# figure out true and pred
true <- c(sector_index)
pred <- c(graph_community$membership)
print(sprintf("The homogenety for Daily data: %s",homogeneity(true, pred)))
print(sprintf("The completness for Daily data: %s",completeness(true,pred)))

QUESTION 5: Report the value of α for the above two cases and provide an interpretation for the difference.

In [ ]:
calculate_alpha(MST,log_return,ticker_sector,unique_sector)